In [0]:
# http://study.mokoron.com/
# https://elibrary.ru/item.asp?id=20399632

In [0]:
!wget 'https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0' positive.csv
!wget 'https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0' negative.csv

--2019-04-26 17:37:47--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.1, 2620:100:6030:1::a27d:5001
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2019-04-26 17:37:48--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc78d5cde0e20ce7938997201bdd.dl.dropboxusercontent.com/cd/0/inline/Afx8d7wGvYs-loSXCtP4DCOgNFIgK5GXzaKSl_ySzZ50yGMtZhRpzlCiLtqUu_cv8Nmui2v9VQ-TBjZGiev6YvRUa6K1PoeUUVoQ_v8JHnj9cgl_0SjZsufNa0W1YTd5XLs/file# [following]
--2019-04-26 17:37:48--  https://uc78d5cde0e20ce7938997201bdd.dl.dropboxusercontent.com/cd/0/inline/Afx8d7wGvYs-loSXCtP4DCOgNFIgK5GXzaKSl_ySzZ50yGMtZhRpzlCiLtqUu_cv8Nmui2v9VQ-TBjZGiev6YvRUa6K1PoeUUVoQ_v8JHn

In [1]:
import pandas as pd

In [2]:
positive = pd.read_csv('positive.csv?dl=0',header=None,encoding='utf-8',delimiter=';')

In [3]:
negative = pd.read_csv('negative.csv?dl=0',header=None,encoding='utf-8',delimiter=';')

In [4]:
positive.shape,negative.shape

((114911, 12), (111923, 12))

In [5]:
positive=positive[[positive.columns[3]]]
negative=negative[[negative.columns[3]]]

In [6]:
positive['target']=1
negative['target']=0

In [7]:
positive.columns=['text','target']
negative.columns=['text','target']

In [8]:
positive.head()

,text,target
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


In [9]:
full_set = pd.concat([positive,negative])

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [11]:
x_train,x_test,y_train,y_test = train_test_split(full_set['text'],full_set.target.values,test_size=0.2,shuffle=True,random_state=2121)

In [12]:
clf = make_pipeline(TfidfVectorizer(ngram_range=(1,2)),MultinomialNB())
clf.fit(x_train,y_train)
preds = clf.predict(x_test)

In [13]:
accuracy_score(y_test,preds)

0.7739546366301496

## Теперь опробуем на этих же данных простейшую сеть с предобученными ембедингами

In [14]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding
from tqdm import tqdm_notebook

Using TensorFlow backend.


In [15]:
f = open('ft_native_300_ru_twitter_nltk_word_tokenize.vec')
embedding_values = {}
for line in tqdm_notebook(f):
    value = line.replace('\n','').split(' ')
    word = value[0]
    coef = np.array(value[1:-1],dtype = 'float32')
    embedding_values[word]=coef

A Jupyter Widget

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(full_set['text'])
sequences_train = tokenizer.texts_to_sequences(x_train)
sequences_test = tokenizer.texts_to_sequences(x_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 291376 unique tokens.


In [17]:
lower=True
embedding_matrix = np.zeros((len(word_index)+1, 100),dtype=np.float32)
for word, i in tqdm_notebook(word_index.items()):
    if lower:
        word = word.lower()
    try:
        embedding_vector = embedding_values[word]
    except:
        embedding_vector = embedding_values["unknown"]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

A Jupyter Widget

In [18]:
from __future__ import print_function
import numpy as np
np.random.seed(1337) 

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, SpatialDropout1D
from keras.layers import LSTM, SimpleRNN, GRU, Bidirectional
from keras.regularizers import l2
from keras.constraints import maxnorm
from keras.optimizers import RMSprop
from keras.datasets import imdb

max_features = 20000
maxlen = 40  
batch_size = 256 

print('Build model...')
model = Sequential()
model.add(Embedding(
    len(word_index)+1,
    100,
    weights=[embedding_matrix],
    input_length=maxlen,
    trainable=False
))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(GRU(32)))
model.add(Dense(1))
model.add(Activation('sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-3,clipnorm=3,clipvalue=3),
              metrics=['accuracy'])

X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)

print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=25,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train...
Instructions for updating:
Use tf.cast instead.
Train on 181467 samples, validate on 45367 samples
Epoch 1/25
181467/181467 [==============================] - 68s 375us/step - loss: 0.5561 - acc: 0.7070 - val_loss: 0.5145 - val_acc: 0.7379
Epoch 2/25
181467/181467 [==============================] - 57s 316us/step - loss: 0.5098 - acc: 0.7403 - val_loss: 0.4798 - val_acc: 0.7615
Epoch 3/25
181467/181467 [==============================] - 64s 352us/step - loss: 0.4922 - acc: 0.7533 - val_loss: 0.4714 - val_acc: 0.7674
Epoch 4/25
181467/181467 [==============================] - 75s 412us/step - loss: 0.4826 - acc: 0.7594 - val_loss: 0.4632 - val_acc: 0.7733
Epoch 5/25
181467/181467 [===========================

In [0]:
# Классический вариант для кэгла
# x = L.SpatialDropout1D(0.2)(x)
# x = L.Bidirectional(L.CuDNNLSTM(64, return_sequences=True))(x)

# avg_pool1 = L.GlobalAveragePooling1D()(x)
# max_pool1 = L.GlobalMaxPooling1D()(x)

# x = L.concatenate([att,avg_pool1, max_pool1])

# preds = L.Dense(1, activation='sigmoid')(x)